In [1]:
%config InlineBackend.figure_format = 'retina'

import pandas
from IPython.display import display, HTML

display(HTML("<style>.container{ width:100% !important; }</style>"))

df = src_df = pandas.read_csv('libavif.csv')
df.sort_values(['effort', 'codec'], inplace=True)
df['size_kb'] = df['size'] / 1000
df['data'] = df.agg('<tt>{0[size_kb]:5.2f}<br>{0[perf]:5.2f}<br>{0[dssim]:7.2f}</tt>'.format, axis=1)

df

,fname,codec,effort,quality,outname,size,enc_time,perf,dssim,size_kb,data
0,munhell,libavif,5,10,munhell.s5.q10,20427,0.370902,0.424012,93.285912,20.427,<tt>20.43<br> 0.42<br> 93.29</tt>
6,munhell,libavif,5,12,munhell.s5.q12,19683,0.398625,0.394524,93.008618,19.683,<tt>19.68<br> 0.39<br> 93.01</tt>
12,munhell,libavif,5,14,munhell.s5.q14,19076,0.371393,0.423452,92.665156,19.076,<tt>19.08<br> 0.42<br> 92.67</tt>
18,munhell,libavif,5,16,munhell.s5.q16,18516,0.416021,0.378026,92.593998,18.516,<tt>18.52<br> 0.38<br> 92.59</tt>
24,munhell,libavif,5,18,munhell.s5.q18,18091,0.425947,0.369217,92.303374,18.091,<tt>18.09<br> 0.37<br> 92.30</tt>
...,...,...,...,...,...,...,...,...,...,...,...
623,parrots,libavif,8,36,parrots.s8.q36,14246,0.057318,6.860257,58.533284,14.246,<tt>14.25<br> 6.86<br> 58.53</tt>
629,parrots,libavif,8,38,parrots.s8.q38,12566,0.049831,7.891043,53.893245,12.566,<tt>12.57<br> 7.89<br> 53.89</tt>
635,parrots,libavif,8,40,parrots.s8.q40,11075,0.047125,8.344173,49.103403,11.075,<tt>11.07<br> 8.34<br> 49.10</tt>
641,parrots,libavif,8,42,parrots.s8.q42,9840,0.050112,7.846779,43.964604,9.840,<tt> 9.84<br> 7.85<br> 43.96</tt>


In [2]:
display(HTML("""<table>
<tr><th>Each cell:
<tr><td>
Size in kb<br>
Megapixels/s<br>
ssimulacra2 mectric
</table>
"""))

tables = []

for fname in src_df.fname.unique():
    df = src_df[src_df.fname == fname]
    df = df.pivot(index='quality', columns=['codec', 'effort'], values='data')
    tables.append(f"<h1>{fname}</h1>" + df.to_html(escape=False))

display(HTML("".join(
    f'<div style="float:left">{table}</div>'
    for table in tables
)))